## Setup

In [1]:
import json
import numpy as np
import os
import random
import sys
import torch
sys.path.append("../..")

from evals.spelling_by_grade import create_full_spelling_prompt, prepare_grade_spelling_eval
from evals.plot_utils import basic_bar_graph, create_table, nested_bar_graph
from evals.eval_utils import ModelType
from evals.eval_list import *
from transformer_lens import HookedTransformer, utils

import plotly.io as pio
pio.renderers.default = "iframe"

words_by_grade = prepare_grade_spelling_eval("../data/GradeSpellingEval.txt", ' ')
words_by_grade[3][:10]

[('able', 'A B L E'),
 ('above', 'A B O V E'),
 ('afraid', 'A F R A I D'),
 ('afternoon', 'A F T E R N O O N'),
 ('again', 'A G A I N'),
 ('age', 'A G E'),
 ('air', 'A I R'),
 ('airplane', 'A I R P L A N E'),
 ('almost', 'A L M O S T'),
 ('alone', 'A L O N E')]

In [2]:
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
# torch.set_grad_enabled(False)

model = HookedTransformer.from_pretrained("pythia-2.8b")
tokenizer = model.tokenizer

n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab

Using pad_token, but it is not set yet.


Loaded pretrained model pythia-2.8b into HookedTransformer


In [3]:
example_prompt = \
"""
Cat: C A T
Dog: D O G
Bird: B I R D
Fish: F I S H
Apple:"""
example_answer = " A"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, prepend_space_to_answer=False)

Tokenized prompt: ['<|endoftext|>', '\n', 'Cat', ':', ' C', ' A', ' T', '\n', 'Dog', ':', ' D', ' O', ' G', '\n', 'B', 'ird', ':', ' B', ' I', ' R', ' D', '\n', 'Fish', ':', ' F', ' I', ' S', ' H', '\n', 'Apple', ':']
Tokenized answer: [' A']


Performance on answer token:
Rank: 0        Logit: 21.78 Prob: 96.88% Token: | A|

Top 0th token. Logit: 21.78 Prob: 96.88% Token: | A|
Top 1th token. Logit: 16.23 Prob:  0.38% Token: | a|
Top 2th token. Logit: 15.72 Prob:  0.23% Token: |  |
Top 3th token. Logit: 15.55 Prob:  0.19% Token: | I|
Top 4th token. Logit: 15.45 Prob:  0.17% Token: | S|
Top 5th token. Logit: 15.41 Prob:  0.17% Token: | O|
Top 6th token. Logit: 15.24 Prob:  0.14% Token: | P|
Top 7th token. Logit: 15.15 Prob:  0.13% Token: | Apple|
Top 8th token. Logit: 15.03 Prob:  0.11% Token: | H|
Top 9th token. Logit: 14.90 Prob:  0.10% Token: | M|


Ranks of the answer tokens: [(' A', 0)]

## DLA

In [4]:
single_shot_eval = GET_FIRST_LETTER_EVAL.run_eval(model, ModelType.TRANSFORMER_LENS, tokenizer, words_by_grade, 5)

Assessing Grade 1


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:04,  2.24it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

 50%|█████     | 2/4 [00:00<00:00,  2.20it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

 75%|███████▌  | 3/4 [00:01<00:00,  2.20it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:01<00:00,  2.26it/s]


In [5]:
prompts = []
for grade in single_shot_eval['data'].keys():
    for item in single_shot_eval['data'][grade]:
        if len(item['word']) > 1:
            prompts.append(item)

len(prompts)

846

In [6]:
import tqdm

# a_proj, e_proj, a_e_proj = torch.zeros(len(prompts), 1058), torch.zeros(len(prompts), 1058), torch.zeros(len(prompts), 1058)

for i, prompt in tqdm.tqdm(enumerate(prompts)):
    logits, cache = model.run_with_cache(prompt['prompt'], prepend_bos=True)
    a_token = model.to_single_token(f" {prompt['word'][0].upper()}")
    e_token = model.to_single_token(f" {prompt['word'][1].upper()}")
    
    a_unembed = model.W_U[:, a_token].detach().cpu()
    e_unembed = model.W_U[:, e_token].detach().cpu()
    
    dif_a_e = a_unembed - e_unembed
    decomp, labels = cache.get_full_resid_decomposition(expand_neurons=False, return_labels=True)
    decomp = decomp[:,0,-1].detach().cpu()
    layer_decomp, layer_labels = cache.accumulated_resid(apply_ln=True, return_labels=True, mlp_input=False, incl_mid=False)
    layer_decomp = layer_decomp[:,0,-1].detach().cpu()


    cur_a, cur_e, cur_a_e = decomp @ a_unembed, decomp @ e_unembed, decomp @ dif_a_e
    
    if i == 0:
        a_proj, e_proj, a_e_proj = cur_a, cur_e, cur_a_e
    else:
        # Keep a rolling average as I go. E.g, 6th item = current * 5/6 + new * 1/6.
        a_proj = a_proj * (i / (i+1)) + (1 / (i+1)) * cur_a
        e_proj = e_proj * (i / (i+1)) + (1 / (i+1)) * cur_e
        a_e_proj = a_e_proj * (i / (i+1)) + (1 / (i+1)) * cur_a_e

    layer_cur_a, layer_cur_e, layer_cur_a_e = layer_decomp @ a_unembed, layer_decomp @ e_unembed, layer_decomp @ dif_a_e
    
    if i == 0:
        layer_a_proj, layer_e_proj, layer_a_e_proj = layer_cur_a, layer_cur_e, layer_cur_a_e
    else:
        # Keep a rolling average as I go. E.g, 6th item = current * 5/6 + new * 1/6.
        layer_a_proj = layer_a_proj * (i / (i+1)) + (1 / (i+1)) * layer_cur_a
        layer_e_proj = layer_e_proj * (i / (i+1)) + (1 / (i+1)) * layer_cur_e
        layer_a_e_proj = layer_a_e_proj * (i / (i+1)) + (1 / (i+1)) * layer_cur_a_e

1it [00:00,  5.57it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


4it [00:00,  7.06it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


6it [00:00,  7.22it/s]

...

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


844it [01:52,  8.26it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


846it [01:53,  7.48it/s]

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


In [7]:
# print(a_proj.shape, e_proj.shape, a_e_proj.shape)
# a_proj, e_proj, a_e_proj = torch.sum(a_proj, 0) / len(prompts), torch.sum(e_proj, 0) / len(prompts), torch.sum(a_e_proj, 0) / len(prompts)
# print(a_proj.shape, e_proj.shape, a_e_proj.shape)

In [8]:
import pandas as pd
import plotly.express as px

decomp_df = pd.DataFrame({
    "labels": labels,
    "a_proj": a_proj,
    "e_proj": e_proj,
    "a_e_proj": a_e_proj,}
)

px.line(decomp_df, x="labels", y=["a_proj", "e_proj", "a_e_proj"], title="Projection of Residuals onto A and E Embeddings")

In [9]:
top_indices = torch.topk(a_e_proj, 10)
print(top_indices)
top_labels = [labels[i] for i in top_indices.indices]
print([(round(top_indices.values[i].item(), 3), labels[top_indices.indices[i]]) for i in range(len(top_labels))])

torch.return_types.topk(
values=tensor([7.9888, 5.6485, 2.8568, 1.4510, 0.8683, 0.7782, 0.6587, 0.6102, 0.5859,
        0.5843]),
indices=tensor([ 575,  517,  913,  553,  424, 1054,  430, 1046,  964, 1050]))
[(7.989, 'L17H31'), (5.649, 'L16H5'), (2.857, 'L28H17'), (1.451, 'L17H9'), (0.868, 'L13H8'), (0.778, '30_mlp_out'), (0.659, 'L13H14'), (0.61, '22_mlp_out'), (0.586, 'L30H4'), (0.584, '26_mlp_out')]


In [10]:
print(model.to_str_tokens(example_prompt))
print(model.to_string(a_token), model.to_string(e_token))

['<|endoftext|>', '\n', 'Cat', ':', ' C', ' A', ' T', '\n', 'Dog', ':', ' D', ' O', ' G', '\n', 'B', 'ird', ':', ' B', ' I', ' R', ' D', '\n', 'Fish', ':', ' F', ' I', ' S', ' H', '\n', 'Apple', ':']
 U  S


In [11]:
layer_decomp_df = pd.DataFrame({
    "labels": layer_labels,
    "a_proj": layer_cur_a,
    "e_proj": layer_cur_e,
    "a_e_proj": layer_cur_a_e,}
)

px.line(layer_decomp_df, x="labels", y=["a_proj", "e_proj", "a_e_proj"], title="Projection of Residuals onto A and E Embeddings By Layer")

In [12]:
clean_prompt = """Cat: C A T
Dog: D O G
Bird: B I R D
Fish: F I S H
Apple:"""
corrupted_prompt = """Cat: C A T
Dog: D O G
Bird: B I R D
Fish: F I S H
Peach:"""

clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

def logits_to_logit_diff(logits, correct_answer=" John", incorrect_answer=" Mary"):
    # model.to_single_token maps a string value of a single token to the token index for that token
    # If the string is not a single token, it raises an error.
    correct_index = model.to_single_token(correct_answer)
    incorrect_index = model.to_single_token(incorrect_answer)
    return logits[0, -1, correct_index] - logits[0, -1, incorrect_index]

# We run on the clean prompt with the cache so we store activations to patch in later.
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = logits_to_logit_diff(clean_logits)
print(f"Clean logit difference: {clean_logit_diff.item():.3f}")

# We don't need to cache on the corrupted prompt.
corrupted_logits = model(corrupted_tokens)
corrupted_logit_diff = logits_to_logit_diff(corrupted_logits)
print(f"Corrupted logit difference: {corrupted_logit_diff.item():.3f}")

Clean logit difference: 1.564
Corrupted logit difference: -0.331


In [13]:
from jaxtyping import Float
from transformer_lens.hook_points import HookPoint
from functools import partial

def imshow(tensor, renderer="iframe", xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

# We define a residual stream patching hook
# We choose to act on the residual stream at the start of the layer, so we call it resid_pre
# The type annotations are a guide to the reader and are not necessary
def residual_stream_patching_hook(
    resid_pre: Float[torch.Tensor, "batch pos d_model"],
    hook: HookPoint,
    position: int
) -> Float[torch.Tensor, "batch pos d_model"]:
    # Each HookPoint has a name attribute giving the name of the hook.
    clean_resid_pre = clean_cache[hook.name]
    resid_pre[:, position, :] = clean_resid_pre[:, position, :]
    return resid_pre

# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
num_positions = len(clean_tokens[0])
patching_result = torch.zeros((model.cfg.n_layers, num_positions), device=model.cfg.device)

for layer in tqdm.tqdm(range(model.cfg.n_layers)):
    for position in range(num_positions):
        # Use functools.partial to create a temporary hook function with the position fixed
        temp_hook_fn = partial(residual_stream_patching_hook, position=position)
        # Run the model with the patching hook
        patched_logits = model.run_with_hooks(corrupted_tokens, fwd_hooks=[
            (utils.get_act_name("resid_pre", layer), temp_hook_fn)
        ])
        # Calculate the logit difference
        patched_logit_diff = logits_to_logit_diff(patched_logits).detach()
        # Store the result, normalizing by the clean and corrupted logit difference so it's between 0 and 1 (ish)
        patching_result[layer, position] = (patched_logit_diff - corrupted_logit_diff)/(clean_logit_diff - corrupted_logit_diff)

token_labels = [f"{token}_{index}" for index, token in enumerate(model.to_str_tokens(clean_tokens))]
imshow(patching_result, x=token_labels, xaxis="Position", yaxis="Layer", title="Normalized Logit Difference After Patching Residual Stream on the Spelling Task")

100%|██████████| 32/32 [01:18<00:00,  2.44s/it]


In [19]:
import nltk
nltk.download('words')

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from probe_training import all_probe_training_runner

# Each probe at 100 epochs takes about 45 minutes on an A6000, so we're only training two.
probe = all_probe_training_runner(model.W_E, model.tokenizer.get_vocab(), criteria_mode="contains_any", alphabet="AB", num_epochs=25)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

...

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

  0%|          | 0/583 [00:00<?, ?it/s]

In [20]:
print(probe)

{'A': LinearProbe(
  (fc): Linear(in_features=2560, out_features=1, bias=True)
), 'B': LinearProbe(
  (fc): Linear(in_features=2560, out_features=1, bias=True)
)}
